In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from opentelemetry.sdk.trace import Tracer
from opentelemetry import trace as trace_api

try:
    trace_api.set_preferred_tracer_implementation(lambda T: Tracer())
    tracer = trace_api.tracer()
except RuntimeError:
    pass

tracer

In [3]:
# https://github.com/open-telemetry/opentelemetry-python/blob/master/ext/opentelemetry-ext-jaeger/examples/jaeger_exporter_example.py

import time
from opentelemetry.ext import jaeger
from opentelemetry.sdk.trace.export import BatchExportSpanProcessor

jaeger_exporter = jaeger.JaegerSpanExporter(
    service_name="kubecon SD 2",
    # configure agent
    agent_host_name="localhost",
    agent_port=6831,
    # optional: configure also collector
    # collector_host_name="localhost",
    # collector_port=14268,
    # collector_endpoint="/api/traces?format=jaeger.thrift",
    # username=xxxx, # optional
    # password=xxxx, # optional
)

# create a BatchExportSpanProcessor and add the exporter to it
span_processor = BatchExportSpanProcessor(jaeger_exporter)

# add to the tracer
tracer.add_span_processor(span_processor)

In [4]:
# https://github.com/opentracing-contrib/python-redis

# import redis
# import redis_opentracing
# from datetime import datetime as dt

# redis_opentracing.init_tracing(shim_tracer)
# client = redis.StrictRedis()
# client

In [5]:
# with shim_tracer.start_active_span('parent') as parent:
#     client.set('foo', 'bar')
#     client.get('foo')

In [6]:
# client.__class__

In [7]:
# client.expire('foo', 10)

In [8]:
# client.get('foo')

In [9]:
"""
This is an example of a library written to work with opentracing-python. It
provides a simple caching decorator backed by Redis, and uses the OpenTracing
Redis integartion to automatically generate spans for each call to Redis.
"""

import json
import pickle
from functools import wraps

import redis
import redis_opentracing


class RedisCache:
    """Redis-backed caching decorator, using OpenTracing!

    Args:
        tracer: an opentracing.tracer.Tracer
    """

    def __init__(self, tracer):
        redis_opentracing.init_tracing(tracer)
        self.tracer = tracer
        self.client = redis.StrictRedis()

    def __call__(self, func):
        @wraps(func)
        def inner(*args, **kwargs):
            with self.tracer.start_active_span("Caching decorator") as scope1:

                # Pickle the call args to get a canonical key. Don't do this in
                # prod!
                key = pickle.dumps((func.__qualname__, args, kwargs))

                pval = self.client.get(key)
                if pval is not None:
                    val = pickle.loads(pval)
                    scope1.span.log_kv(
                        {"msg": "Found cached value", "val": val}
                    )
                    return val

                scope1.span.log_kv({"msg": "Cache miss, calling function"})
                with self.tracer.start_active_span(
                    'Call "{}"'.format(func.__name__)
                ) as scope2:
                    scope2.span.set_tag("func", func.__name__)
                    scope2.span.set_tag("args", str(args))
                    scope2.span.set_tag("kwargs", str(kwargs))

                    val = func(*args, **kwargs)
                    scope2.span.set_tag("val", str(val))

                # Let keys expire after 10 seconds
                self.client.setex(key, 10, pickle.dumps(val))
                return val

        return inner


In [10]:
span.set_attribute?

Object `span.set_attribute` not found.


In [11]:
from opentelemetry.ext import opentracing_shim

opentracing_tracer = opentracing_shim.create_tracer(tracer)
redis_cache = RedisCache(opentracing_tracer)


@redis_cache
def fib(n):
    if n < 0:
        raise ValueError
    if n == 0 or n == 1:
        return 1
    return fib(n - 1) + fib(n - 2)

In [16]:
with tracer.start_as_current_span('Fibonacci') as span:
    span.set_attribute("is_example", "yes :)")
    fib(4)

In [15]:
span.set_attribute?

In [13]:
span.events

BoundedList([], maxlen=128)

In [14]:
from opentelemetry.ext import jaeger as jaeger_ext

In [15]:
jaeger_ext._extract_logs_from_span(span)